In [1]:
import json
import os
import re
from datetime import datetime, timedelta
from urllib.parse import urlparse
import time
import yaml
from datetime import date
import dotenv

import pandas as pd 

import sqlite3

import pyperclip

import bs4
from bs4 import BeautifulSoup
import requests
import tldextract

from openai import OpenAI
from pydantic import BaseModel, Field
from typing import List

import IPython
from IPython.display import HTML, Markdown, display
import markdown

from atproto import Client

import PIL
from PIL import Image

# disable 'show more outputs' nonsense
from IPython.core.interactiveshell import InteractiveShell

# Always display all outputs in a cell
InteractiveShell.ast_node_interactivity = "all"


/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'default' attribute with value None was provided to the `Field()` function, which has no effect in the context it was used. 'default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


In [2]:
print(f"requests        {requests.__version__}")
print(f"BeautifulSoup   {bs4.__version__}")


requests        2.32.5
BeautifulSoup   4.14.2


In [3]:
dotenv.load_dotenv()


True

# Load posts from BlueSky and format for Substack or a blog post
for now I share the interesting stuff on bluesky and then use this code to grab latest BlueSky 'tweets' and format a [Substack post](https://skynetandchill.com)


In [4]:
os.environ['BSKY_USERNAME']

'druce.ai'

In [5]:
client = Client(base_url='https://bsky.social')
client.login(os.environ['BSKY_USERNAME'], os.environ['BSKY_SECRET'])

mydid = {"did":"did:plc:qomkdnxrqw3gkbytdxea5z65"}

data = client.get_author_feed(
    actor=mydid['did'],
    filter='posts_and_author_threads',
    limit=80,
)


ProfileViewDetailed(did='did:plc:qomkdnxrqw3gkbytdxea5z65', handle='druce.ai', associated=ProfileAssociated(activity_subscription=ProfileAssociatedActivitySubscription(allow_subscriptions='followers', py_type='app.bsky.actor.defs#profileAssociatedActivitySubscription'), chat=ProfileAssociatedChat(allow_incoming='following', py_type='app.bsky.actor.defs#profileAssociatedChat'), feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreihi55ddvmohg6dztiwlkwdbts5fd76iaenef2sriacvj7ibtezagi@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:qomkdnxrqw3gkbytdxea5z65/bafkreigu723bmkba3mff4krhcrrf7b5zw2svixwbvtj54v5ho64o2xpmzu@jpeg', created_at='2024-02-12T00:15:55.099Z', description='Daily artificial intelligence news, lovingly curated by man and machine. https://www.skynetandchill.com\n\nNeo-Luddite AI maven. On a long enough timeline, p(doom) for every

In [6]:
def remove_urls(text):
    # Regular expression to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    # Substitute found URLs with an empty string
    clean_text = re.sub(url_pattern, '', text)
    return clean_text


In [7]:
def rawfetchurl(url, timeout=60):
    """get url using requests with specified timeout. return response object, status, content-type"""
    try:
        response = requests.get(url, timeout=timeout)
    except httplib.BadStatusLine:
        log("Bad response (?) fetching url %s " % url)
        response = None
    except requests.Timeout:
        log("Timeout fetching url %s " % url)
        response = None
    except requests.ConnectionError as e:
        log("Connection error (%s) fetching url %s " % (str(e), url))
        response = None
    except requests.TooManyRedirects:
        log("Too many redirects fetching url %s " % url)
        response = None
    except requests.exceptions.MissingSchema:
        log("Missing schema url %s " % url)
        response = None
    except requests.exceptions.InvalidSchema:
        log("Invalid schema url %s " % url)
        response = None
    except requests.exceptions.InvalidURL as e:
        log("Invalid url %s, %s" % (url, str(e)))
        response = None
    except ValueError as e:
        # don't log url because possibly malformed url
        log("ValueError, url ?: ? ")
        response = None
    except httplib.IncompleteRead as e:
        log("IncompleteRead, url %s: %s " % (url, str(e)))
        response = None
    except urllib3.exceptions.SSLError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ContentDecodingError as e:
        log("SSLError, url %s: %s " % (url, str(e)))
        response = None
    except requests.exceptions.ChunkedEncodingError as e:
        log("ChunkedEncodingError, url %s: %s " % (url, str(e)))
        response = None
    except UnicodeEncodeError as e:
        log("UnicodeEncodeError, url %s: %s " % (url, str(e)))
        response = None
    except OpenSSL.SSL.SysCallError as e:
        log("OpenSSL.SSL.SysCallError, url %s: %s " % (url, str(e)))
        response = -1
    except OpenSSL.SSL.ZeroReturnError as e:
        log("OpenSSL.SSL.ZeroReturnError, url %s: %s " % (url, str(e)))
        response = -1

    # except requests.packages.urllib3.exceptions.DecodeError as e:
    #     utilLog("DecodeError, url %s: %s " % (url, str(e)))
    #     response = None

    return response



In [8]:
def resize_and_crop(input_image_path, output_image_path, desired_height=240):
    # Load the image
    with Image.open(input_image_path) as img:
        img = img.convert('RGB')

        # Calculate the new width maintaining the aspect ratio
        aspect_ratio = img.width / img.height
        new_width = int(desired_height * aspect_ratio)

        # Resize the image
        resized_img = img.resize((new_width, desired_height))

        # Save the resized image
        resized_img.save(output_image_path)


In [9]:
# attempt to remove traiing inline URLs

def truncate_last_occurrence(text: str) -> str:
    # Find trailing occurrence of a space followed by any sequence of characters followed by 3 periods
    pattern = r'\s+\S+\.{3}$'
    return re.sub(pattern, '', text)

# Example text for testing

example_text = """Elon Musk says we'll run out of power capacity to run all the AI chips in 2025
newatlas.com/technology/e..."""

# Truncate the last occurrence
print(truncate_last_occurrence(example_text))


Elon Musk says we'll run out of power capacity to run all the AI chips in 2025


In [10]:
def get_og_tags(url):
    """get a dict of Open Graph og: tags such as title in the HEAD of a URL"""
    retdict = {}
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            head = soup.head
            if head:
                og_tags = head.find_all(
                    property=lambda prop: prop and prop.startswith("og:")
                )
                for tag in og_tags:
                    if "content" in tag.attrs:
                        retdict[tag["property"]] = tag["content"]

                page_title = ""
                title_tag = soup.find("title")
                if title_tag:
                    page_title = title_tag.text
                    if page_title:
                        retdict["title"] = page_title
        return retdict
    except requests.RequestException as e:
        log(f"Error fetching {url}: {e}")
    return retdict


url = "https://druce.ai"
get_og_tags(url)

{'og:title': 'Druce.ai',
 'og:locale': 'en_US',
 'og:description': 'A Blog on AI, Tech, Markets and Random Stuff, by Druce Vertes',
 'og:url': 'https://druce.ai/',
 'og:site_name': 'Druce.ai',
 'og:type': 'website',
 'title': 'Druce.ai | A Blog on AI, Tech, Markets and Random Stuff, by Druce Vertes'}

In [11]:
def delete_files(outputdir):

    # Iterate over all files in the directory
    for filename in os.listdir(outputdir):
        if filename.startswith('.'):
            continue
        file_path = os.path.join(outputdir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)  # Remove the file
            elif os.path.isdir(file_path):
                # If you want to remove subdirectories as well, use os.rmdir() here
                pass
        except Exception as e:
            log(f'Failed to delete {file_path}. Reason: {e}')
            

In [12]:
imgdir = 'download/images'  # for images
delete_files(imgdir)

conn = sqlite3.connect('newsletter_agent.db')
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.domain_name: row.site_name for row in sites_df.itertuples()}
conn.close()


In [13]:
!mv most_recent_bsky_20251231.md most_recent_bsky.md



mv: most_recent_bsky_20251231.md: No such file or directory


In [14]:
with open("most_recent_bsky.md", 'r') as file:
    prev_start = file.readline().strip()    
prev_start

'Turning AI into your personal productivity assistant. One study found 8% of AI users created more work, so firms should pilot select productivity tools like Shortwave or Otter.'

In [15]:
today_str = date.today().strftime("%Y%m%d")
os.rename("most_recent_bsky.md", f"most_recent_bsky_{today_str}.md")


In [16]:
def parse_feed(prev_start=""):
    display_dict = {}
    image_dict = {}
    for i, post in enumerate(data.feed):
        
        # check date
        date_str = data.feed[0].post.record.created_at
        dt = datetime.fromisoformat(date_str.rstrip('Z'))
        # break if more than 30 hours ago
        if dt  < datetime.now() - timedelta(hours=30):
            break

        # get post_str, post_url
        post_str = post.post.record.text.strip()
        post_str = truncate_last_occurrence(post_str)
        post_str = post_str.replace("$", "\\\$")  # so Markdown doesn't interpret $ as latex escape
        
        if i==0:
            with open("most_recent_bsky.md", 'w') as file:
                file.write(post_str)
                
        # stop if we are up to previous
        if post_str == prev_start:
            print(f"matched previous value at i={i}")
            break

        try:
            post_url = post.post.record.embed.external.uri.rstrip()
        except:
            post_url = ""

        if post_url:
            print(post_url) # to know if it's blocked
            # get site name , image using og tags
            try:
                tag_dict = get_og_tags(post_url)
            except:
                tag_dict = {}

            site_name = tag_dict.get('og:site_name')
            img_url = tag_dict.get('og:image')

            # get site_name from stored values as backup
            if not(site_name):
                extracted = tldextract.extract(str(post_url))
                if extracted.domain:
                    domain = f"{extracted.domain}.{extracted.suffix}"
                else:  # local hosts, ip addresses or other edge cases
                    domain = extracted.subdomain
                domain=domain.lower()
                site_name = sites_dict.get(domain, domain)

            md_str = f"[{post_str}]({post_url})  - *{site_name}*"

            display_dict[i]=md_str

            # write local copy of image
            if img_url:
                try:
                    r = rawfetchurl(img_url)
                    content_type = r.headers['Content-Type']
                    content_type = content_type[content_type.find('/')+1:]
                    impath = f"{imgdir}/source{i}.{content_type}"
                    with open(impath, 'wb') as file:
                        file.write(r.content)
                    output_path = f'{imgdir}/Image{i}.jpg'
                    resize_and_crop(impath, output_path)
                    image_dict[i] = f"![image]({output_path})\n\n"
                except Exception as e:
                    print(e)

        else: # no url
            md_str = f"\n{post_str}" + "\n\n"
            display_dict[i]=md_str
            
    return display_dict, image_dict

display_dict, image_dict = parse_feed(prev_start)
len(display_dict)


https://www.youtube.com/watch?v=MiUHjLxm3V0
https://www.theverge.com/tech/854112/gemini-google-tv-nano-banana-veo-ces
https://snscratchpad.com/posts/looking-ahead-2026/
https://www.motor1.com/news/783375/chevrolet-tahoe-chatgpt-visor-malfunction/
https://www.wsj.com/tech/ai/hospitals-are-a-proving-ground-for-what-ai-can-do-and-what-it-cant-60e4020c
https://www.wired.com/story/ai-deepfakes-are-impersonating-pastors-to-try-and-scam-their-congregations/
https://github.com/NirDiamant/GenAI_Agents
https://github.com/nibzard/awesome-agentic-patterns
cannot identify image file 'download/images/source7.html; charset=utf-8'
https://www.ft.com/content/d36f3392-9a73-476a-9357-8ff311bb04da
https://www.theverge.com/tech/851393/plaud-notepin-s-desktop-app-ces
https://gizmodo.com/sorry-tamagotchi-fans-its-ai-time-2000705349
https://www.memeorandum.com/260105/p10#a260105p10
https://www.wsj.com/politics/policy/ice-facial-recognition-app-mobile-fortify-dfdd00bf
https://www.businessinsider.com/nyu-profes

28

In [17]:
output_str = ""
for i in sorted(display_dict.keys()):
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

# display(Markdown(output_str))


In [18]:
stop_index = 999

In [19]:
output_str = ""
for i in sorted(display_dict.keys()):
    if i >= stop_index:
        break
    md_str = f"{i}. "
    md_str += display_dict.get(i)
    if image_dict.get(i):
        md_str += image_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str

output_str += f"[Created with ❤️ and AI by druce.ai](https://github.com/druce/OpenAIAgentsSDK)"
output_str += f"Follow the latest AI headlines as they are discovered via [druce.ai on Bluesky](https://bsky.app/profile/druce.ai)"

# display(Markdown(output_str))

In [20]:
output_str =""
markdown_extensions = [
    # 'tables',
    # 'fenced_code',
    # 'codehilite',
    'attr_list',
    'def_list',
    # 'footnotes',
    'markdown.extensions.nl2br',
    'markdown.extensions.sane_lists'
]

output_str = ""
output_array = []
for i in sorted(display_dict.keys()):
    md_str=""
    if i >= stop_index:
        break00
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    output_array.append(display_dict.get(i))
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

output_df = pd.DataFrame(output_array)
output_df.columns = ['markdown']
output_df = output_df.reset_index()
    
# display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
        f.write(html_str)

9096

In [21]:
# Define Pydantic model for structured output
class ItemList(BaseModel):
    """List of story indexes"""
    items: List[int] = Field(description="The ordered list of story indexes")

# Your prompts
system_prompt = """
# ROLE
You are a newsletter editor.

# TASK
Group and reorder the items in the Markdown newsletter supplied below so that:
1. Items that cover similar companies, individuals, technologies or topics are **next to each other** (i.e., logically grouped).
2. Groups appear from **most important** to **least important** according to the **IMPORTANCE FACTORS** listed below.
3. Importance of groups is determined by the most important story 

# HOW TO GROUP
- If two or more items discuss the same organization, person, technology, or topic, keep them all together.
- Within any group, order by importance using **IMPORTANCE FACTORS**.

# HOW TO ORDER
- Lead with stories that have highest importance per the **IMPORTANCE FACTORS** below
- Follow with narrower‑scope or opinion pieces.
- End with lighter or human‑interest items.

# IMPORTANCE FACTORS
1 Magnitude of impact : large user base, $ at stake, broad social reach
2 Novelty : breaks conceptual ground, not a minor iteration
3 Authority : reputable institution, peer-review, regulatory filing, on-record executive
4 Verifiability : code, data, benchmarks, or other concrete evidence provided
5 Timeliness : early signal of an important trend or shift
6 Breadth : cross-industry / cross-disciplinary / international implications
7 Strategic consequence : shifts competitive, power, or policy dynamics
8 Financial materiality : clear valuation or growth or revenue impact
9 Risk & safety : raises or mitigates critical alignment, security, or ethical risk
10 Actionability : informs concrete decisions for investors, policymakers, practitioners
11 Longevity : likely to matter or be referred to in coming days, weeks, or months
12 Independent corroboration : confirmed by multiple sources or datasets
13 Clarity : enough technical/context detail to judge merit; minimal hype

# INPUT FORMAT
You will receive a JSON array of items, with an index and markdown summary of the item.

# OUTPUT FORMAT
- Return **only** the reordered indexes using the provided schema.
- Do **not** alter the wording of individual items (other than correcting spelling and punctuation, and normalizing spaces or line breaks as needed).
- Insert one blank line between each logical group for readability.
- Do not output anything other than the newsletter itself—no headers, explanations, or commentary.

"""

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Make the API call with structured output
completion = client.beta.chat.completions.parse(
    model="gpt-5.2",
    reasoning_effort="medium",
    response_format=ItemList,    
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": output_df.to_json(orient="records")}
    ],
)

# Extract the parsed response
response = completion.choices[0].message.parsed

# Access the result
print(len(response.items)) # This will be a list of integers


28


In [22]:
output_str = ""
for i in response.items:
    md_str = ""
#     output_str += f"{i}\n\n"
    if image_dict.get(i):
        md_str += "\n\n"
        md_str += image_dict.get(i)
    md_str += display_dict.get(i)
    md_str += "\n\n---\n\n"
    output_str += md_str
    
output_str += f"Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)"

display(Markdown(output_str))
html_str = markdown.markdown(output_str, extensions=markdown_extensions)

with open('skynet.html', 'w', encoding="utf-8") as f:
    f.write(html_str)
    

[Samsung co-CEO TM Roh says the company rolled out Gemini AI features to ~400M mobile devices so far and aims to hit 800M in 2026](https://www.reuters.com/world/china/samsung-double-mobile-devices-powered-by-googles-gemini-800-mln-units-this-year-2026-01-05/)  - *Reuters*

---

[Smart TVs get AI to edit photos and create videos](https://www.theverge.com/tech/854112/gemini-google-tv-nano-banana-veo-ces)  - *The Verge*

---



![image](download/images/Image23.jpg)

[At CES 2026, integrated chatbots and built-in AI are now commonplace, and companies must focus on refining user experience to succeed in the AI era.](https://www.wired.com/story/ces-2026-what-to-expect/)  - *WIRED*

---



![image](download/images/Image10.jpg)

[Takway.Ai unveiled Sweekar at CES 2026, a Tamagotchi-style AI egg that uses Google Gemini and ChatGPT for memory and interaction, offers growth stages, and will launch a March Kickstarter priced \\$150–\\$200, raising child-safety and privacy concerns.](https://gizmodo.com/sorry-tamagotchi-fans-its-ai-time-2000705349)  - *Gizmodo*

---

[Plaud launched the \\$179 NotePin S, replacing the 2024 NotePin's haptic squeeze with a physical recording/highlight button and bundling accessories, and released Plaud Desktop for Mac and PC to capture meeting audio, free for Plaud Note or NotePin owners.](https://www.theverge.com/tech/851393/plaud-notepin-s-desktop-app-ces)  - *The Verge*

---

[Venture capital flowed heavily to AI in 2025: more than half of VC dollars and 36% of deals went to AI, and Crunchbase reports 14% of funding went to OpenAI and Anthropic.](https://www.fastcompany.com/91465347/2026-venture-capital-artificial-intelligence-openai-anduril)  - *Fast Company*

---



![image](download/images/Image8.jpg)

[Surging AI investment may repeat past cycles of over-optimism and heavy capital spending, posing risks for investors.](https://www.ft.com/content/d36f3392-9a73-476a-9357-8ff311bb04da)  - *Financial Times*

---

[Asset managers warn heavy corporate AI investment could drive inflation into 2026–27 and force central banks to halt rate cuts, and Morgan Stanley's Andrew Sheets forecasts US CPI will stay above 2% through end‑2027.](https://finance.yahoo.com/news/analysis-ai-driven-inflation-2026s-060212201.html)  - *Yahoo*

---



![image](download/images/Image11.jpg)

[David Atkins argues AI's rapid automation is concentrating economic returns among top firms and investors, amplifying winner-take-all dynamics and prompting calls for wealth taxes, stronger antitrust and AI governance to address rising inequality.](https://www.memeorandum.com/260105/p10#a260105p10)  - *memeorandum*

---



![image](download/images/Image2.jpg)

[Satya Nadella: the tech industry needs to move past “slop vs sophistication” arguments and view AI as a “cognitive amplifier”, not a human substitute](https://snscratchpad.com/posts/looking-ahead-2026/)  - *sn scratchpad*

---



![image](download/images/Image24.jpg)

[AI start-ups take on Google in fight to reshape web browser market](https://www.ft.com/content/5d566029-6aee-4627-a665-81108a1eb70e)  - *Financial Times*

---

[U.S. Immigration and Customs Enforcement is using facial recognition technology to rapidly identify and arrest people.](https://www.wsj.com/politics/policy/ice-facial-recognition-app-mobile-fortify-dfdd00bf)  - *The Wall Street Journal*

---



![image](download/images/Image25.jpg)

[NYC Wegmans implements facial recognition for security](https://gothamist.com/news/nyc-wegmans-is-storing-biometric-data-on-shoppers-eyes-voices-and-faces)  - *Gothamist*

---



![image](download/images/Image14.jpg)

[Boston Dynamics is training Atlas, an all-electric Nvidia-powered humanoid, to perform factory tasks using VR teleoperation, motion-capture demonstrations and large-scale simulation (over 4,000 simulated Atlases trained six hours), tested autonomously sorting roof racks at Hyundai's Georgia plant.](https://www.cbsnews.com/news/boston-dynamics-training-ai-humanoids-to-perform-human-jobs-60-minutes/)  - *CBS News*

---



![image](download/images/Image0.jpg)

[ASML: The Ridiculous Engineering Of The World's Most Important Machine](https://www.youtube.com/watch?v=MiUHjLxm3V0)  - *YouTube*

---

[OpenAI's new report says 40 million Americans use ChatGPT for health care, employing chatbots to navigate insurance, find providers and perform basic triage, a trend that could prompt partnerships among AI vendors, health systems and insurers.](https://www.axios.com/2026/01/05/chatgpt-openai-health-insurance-aca)  - *Axios*

---

[Hospitals are serving as proving grounds for artificial intelligence, demonstrating both its potential uses and its current limitations.](https://www.wsj.com/tech/ai/hospitals-are-a-proving-ground-for-what-ai-can-do-and-what-it-cant-60e4020c)  - *The Wall Street Journal*

---

[Alibaba's PANDA AI tool has analyzed 180,000 CT scans and detected about 24 pancreatic cancer cases since its deployment in November 2024, a Chinese hospital said.](https://www.nytimes.com/2026/01/02/world/asia/china-ai-cancer-pancreatic.html)  - *The New York Times*

---

[Nitin Ware, lead engineer at Salesforce’s AI platform, warns agentic AI that autonomously executes tasks like scheduling and billing creates legal liability, fraud and trust risks for platforms, vendors and consumers and requires governance, audit trails and accountability mechanisms.](https://www.washingtonpost.com/opinions/2026/01/05/agentic-artificial-intelligence-ai-tech/)  - *The Washington Post*

---

[Physicist Max Tegmark, who advocates pausing AGI development, was subpoenaed by OpenAI regarding the Future of Life Institute's past ties to Elon Musk.](https://www.wsj.com/tech/ai/who-is-max-tegmark-future-of-life-institute-accffffc)  - *The Wall Street Journal*

---

[A curated catalog of agentic AI patterns documents practical techniques—task decomposition, tool orchestration, memory management, self‑healing feedback, CI code‑review loops, multimodal integration, and security controls—and aims to evolve as contributors apply and extend it.](https://github.com/nibzard/awesome-agentic-patterns)  - *GitHub*

---



![image](download/images/Image6.jpg)

[See also Nir Diamant](https://github.com/NirDiamant/GenAI_Agents)  - *GitHub*

---

[A developer runs six Claude Code agents in parallel from an iPhone via Termius connected to a Vultr vhf-8c-32gb VM (\\$0.29/hr), using Tailscale, tmux/mosh for persistence and Poke webhooks to review PRs, refactor, and fix bugs.](https://granda.org/en/2026/01/02/claude-code-on-the-go/)  - *granda*

---



![image](download/images/Image26.jpg)

[first thing to ask your ai when you vibecode is how to set up a sandbox and back up your data](https://www.reddit.com/media?url=https%3A%2F%2Fi.redd.it%2Fij72c23e6aag1.jpeg)  - *Reddit*

---



![image](download/images/Image5.jpg)

[AI-generated deepfakes are impersonating U.S. pastors to broadcast incendiary sermons and solicit donations from their congregations.](https://www.wired.com/story/ai-deepfakes-are-impersonating-pastors-to-try-and-scam-their-congregations/)  - *WIRED*

---



![image](download/images/Image3.jpg)

[Sarah Novak replaced a driver's-side visor on her 2020 Chevrolet Tahoe using ChatGPT instructions, after which the SUV flashed warnings and refused to start despite a nearly new \\$400 battery, with generic AI troubleshooting unable to clear BCM-related fault codes.](https://www.motor1.com/news/783375/chevrolet-tahoe-chatgpt-visor-malfunction/)  - *Motor1.com*

---



![image](download/images/Image13.jpg)

[Panos Ipeirotis, an NYU Stern professor, built an AI oral-exam system using ElevenLabs speech to conduct two-part ~25-minute interviews for 36 students over nine days at about \\$15 compute cost, auto-graded by a council of LLMs (Claude, Gemini, ChatGPT).](https://www.businessinsider.com/nyu-professor-ai-oral-exam-mckinsey-memo-business-school-2026-1)  - *Business Insider*

---


With apologies to Woody Guthrie



---

Follow the latest AI headlines via [SkynetAndChill.com on Bluesky](https://bsky.app/profile/skynetandchill.com)

9096

In [26]:
pyperclip.copy( output_str.replace("\\\$","$"))


In [24]:
# Define Pydantic model for structured output
class ItemList(BaseModel):
    """List of story indexes"""
    items: List[str] = Field(description="The list of proposed headlines")

system_prompt = """
You are a seasoned culture-tech editor writing a witty, irreverent newsletter about the AI industry (think: The Verge meets John Oliver).

You’ll receive a batch of article headlines and summaries. Based on them, generate short, funny, pun-heavy, or referential section headlines that are:
	•	1–7 words long
	•	Playful, sharp, and meme-literate
	•	Can mix puns, pop culture, business jargon, and double meaning
	•	Each headline should capture the essence or irony of the news item
	•	You can optionally include a 1-sentence tagline/sub-hed explaining the joke or tone

Examples of tone and style:
	•	“Silicon soulmates?”
	•	“Chip Happens”
	•	“Rogue Agents of Change”
	•	“GPT and the Terrible, Horrible, No Good, Very Bad PR Week”
	•	“Mo Money, Mo Compute”
	•	“Will OpenAI ever appoint a robot Sam Alt-man as CEO?”
	•	“Bend it like Bengio”
"""

# Make the API call with structured output
completion = client.beta.chat.completions.parse(
    model="gpt-5.1",
    reasoning_effort="medium",
    response_format=ItemList,    
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": output_str.replace("\\\$","$")}
    ],
)

# Extract the parsed response
response = completion.choices[0].message.parsed

# Access the result
print(response) # This will be a list of integers


items=['Gemini Galaxy Quest', 'Netflix And Skill', 'CES: Chatbots In Every Pot', 'Yolk Of The Machine', 'Minutes To Winnovation', 'Series A.I. Funding Frenzy', 'Irrational ExuberAItance', 'Promptflation Nation', 'Winner-Take-All.exe', 'Cognitive Amplifier, Not SLOPbot', 'Browser Wars: Phantom MenAIs', 'ICE, ICE, Surveillance', 'Face ID, But Make It Groceries', 'Factory Settings: Humanoid', 'The Lithography Boss Level', 'WebMD, But Make It GPT', 'ICU, AI See You', 'PANDA Express Diagnosis', 'Move Fast And Break Compliance', 'Subpoena, But Make It Existential', 'Design Patterns For Rogue Agents', 'Choose Your Fighter: AI Agents', 'Six Coders And An iPhone', 'First, Do No Data Harm', 'Thou Shalt Not Catfish', 'Check Engine, Not ChatGPT', 'Tell Me About Yourself, To The Robot']


In [25]:
for item in response.items:
    display(Markdown(f"### {item}"))

### Gemini Galaxy Quest

### Netflix And Skill

### CES: Chatbots In Every Pot

### Yolk Of The Machine

### Minutes To Winnovation

...

### Six Coders And An iPhone

### First, Do No Data Harm

### Thou Shalt Not Catfish

### Check Engine, Not ChatGPT

### Tell Me About Yourself, To The Robot